In [1]:
import pandas as pd
import json
from yelpapi import YelpAPI
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
#Getting the api login info
with open('/Users/alicechen/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

In [3]:
#What I am looking up 
location = 'Los Angeles, California'
term = 'coffee'

In [4]:
# Specify folder for saving data
FOLDER = "Data/"
os.makedirs(FOLDER,exist_ok=True)

In [5]:
location.split(',')[0]

'Los Angeles'

In [6]:
#Specifying JSON_FILE filename 
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [7]:
JSON_FILE

'Data/Los Angeles-coffee.json'

In [8]:
"""This function delete JSON_FILE then create a new one. """
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f) 

In [10]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=location,
                                term=term,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/Los Angeles-coffee.json already exists. Deleting previous file...
[i] Data/Los Angeles-coffee.json not found. Saving empty list to new file.
- 0 previous results found.


610

In [11]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=location,
                                    term=term, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.1)

  0%|          | 0/610 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [13]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,pWzU4ikhUFnBC6rg9iq8_w,alibi-coffee-los-angeles-5,Alibi Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/zatNnq...,False,https://www.yelp.com/biz/alibi-coffee-los-ange...,201,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 34.08325, 'longitude': -118.32566}","[pickup, delivery]","{'address1': '5770 Melrose Ave', 'address2': '...",+13238484523,(323) 848-4523,2433.766398,NaN
1,Re-YQCm7QFt-suXx4CSlAw,document-coffee-bar-los-angeles-3,Document Coffee Bar,https://s3-media1.fl.yelpcdn.com/bphoto/m-lDXC...,False,https://www.yelp.com/biz/document-coffee-bar-l...,645,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 34.0612100152212, 'longitude': -1...",[delivery],"{'address1': '3850 Wilshire Blvd', 'address2':...",,,948.222799,$$
2,KWuSIrgx9fx2m8JuqknZPA,3thyme-coffee-and-herbs-los-angeles-2,3THYME Coffee & Herbs,https://s3-media2.fl.yelpcdn.com/bphoto/tO7wxh...,False,https://www.yelp.com/biz/3thyme-coffee-and-her...,77,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 34.06322, 'longitude': -118.30393}",[],"{'address1': '600 S Harvard Blvd', 'address2':...",+12135683435,(213) 568-3435,1628.440379,$$
3,3sBu25RsAszfk9-50tJoIQ,neighborhood-los-angeles-2,Neighborhood,https://s3-media2.fl.yelpcdn.com/bphoto/jbAJTy...,False,https://www.yelp.com/biz/neighborhood-los-ange...,122,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 34.07258233, 'longitude': -118.34...",[delivery],"{'address1': '133 S La Brea Ave', 'address2': ...",+13234334152,(323) 433-4152,2433.029391,NaN
4,gnRQ4d8RdMpr0wbhVBjEng,coffee-mco-los-angeles,Coffee MCO,https://s3-media2.fl.yelpcdn.com/bphoto/w_MuU2...,False,https://www.yelp.com/biz/coffee-mco-los-angele...,523,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 34.05218, 'longitude': -118.28737}",[delivery],"{'address1': '2580 W Olympic Blvd', 'address2'...",+12136747381,(213) 674-7381,3290.917541,$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
995,8dCe67uDX1S3XdW383b8KQ,lard-lad-donuts-universal-city-2,Lard Lad Donuts,https://s3-media1.fl.yelpcdn.com/bphoto/xL7NdL...,False,https://www.yelp.com/biz/lard-lad-donuts-unive...,207,"[{'alias': 'donuts', 'title': 'Donuts'}]",4.5,"{'latitude': 34.1391376513675, 'longitude': -1...",[],"{'address1': '100 Universal City Plz', 'addres...",+18008648377,(800) 864-8377,9142.811509,$$
996,1cOWdny9861vAXEuzCHIIQ,little-ground-cafe-south-pasadena-2,Little Ground Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/o4V7Km...,False,https://www.yelp.com/biz/little-ground-cafe-so...,2,"[{'alias': 'breakfast_brunch', 'title': 'Break...",5.0,"{'latitude': 34.11463889946774, 'longitude': -...",[],"{'address1': '1009 El Centro St', 'address2': ...",,,16268.018953,NaN
997,-ZC3RdkKMe0AEvRdn3fNBA,drnk-coffee-tea-los-angeles-18,Drnk Coffee + Tea,https://s3-media3.fl.yelpcdn.com/bphoto/45OvR5...,False,https://www.yelp.com/biz/drnk-coffee-tea-los-a...,16,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 34.0318784, 'longitude': -118.284...",[delivery],"{'address1': '2595 S Hoover St', 'address2': '...",+12136251345,(213) 625-1345,4735.356265,NaN
998,zXEIeMnjX9GAJp63zbAuBw,sweetsalt-toluca-lake-2,Sweetsalt,https://s3-media1.fl.yelpcdn.com/bphoto/q5WNwF...,False,https://www.yelp.com/biz/sweetsalt-toluca-lake...,942,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 34.15191, 'longitude': -118.35348}","[delivery, pickup]","{'address1': '10218 Riverside Dr', 'address2':...",+18185097790,(818) 509-7790,10479.520835,$$
999,DjlSNpl726qtV9rXjCU-Yg,spring-st-cafe-los-angeles-2,Spring St. Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/l3Uapx...,False,https://www.yelp.com/biz/spring-st-cafe-los-an...,79,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 34.047747455049, 'longitude': -11...","[delivery, pickup]","{'address1': '453 S Spring St', 'address2': 'F...",+12135370900,(213) 537-0900,6789.303874,$$
